In [1]:
%env CUDA_VISIBLE_DEVICES=4
%env GRB_LICENSE_FILE=/home/yppatel/gurobi.lic

env: CUDA_VISIBLE_DEVICES=4
env: GRB_LICENSE_FILE=/home/yppatel/gurobi.lic


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import osmnx as ox
import networkx as nx

In [3]:
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [58]:
cal_true_traffics = np.load("cal_true_traffics.npy")
cal_pred_traffics = np.load("cal_pred_traffics.npy")

test_true_traffics = np.load("test_true_traffics.npy")
test_pred_traffics = np.load("test_pred_traffics.npy")

In [59]:
shuffle_sets = False
if shuffle_sets:
    inds = np.random.choice(256, 256, replace=False)

    true_traffics = np.vstack([cal_true_traffics, test_true_traffics])
    pred_traffics = np.concatenate([cal_pred_traffics, test_pred_traffics], axis=1)

    cal_size = 150

    cal_true_traffics, cal_pred_traffics = true_traffics[inds[:cal_size]], pred_traffics[:,inds[:cal_size]]
    test_true_traffics, test_pred_traffics = true_traffics[inds[cal_size:]], pred_traffics[:,inds[cal_size:]]

In [60]:
G = ox.graph_from_place("Manhattan, New York City, New York", network_type="drive")

G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

edges = ox.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60}
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)

In [61]:
A = nx.incidence_matrix(G, oriented=True).todense()
b = np.zeros(len(G.nodes)) # b entries: 1 for source, -1 for target, 0 o.w.
b[1]   = -1
b[120] = 1 

In [66]:
sample_idx = 0

alpha = 0.05
box_cal_scores = np.linalg.norm(cal_true_traffics - cal_pred_traffics[sample_idx], np.inf, axis=1)
conformal_quantile = np.quantile(box_cal_scores, q=1 - alpha, axis=0)

box_true_scores = np.linalg.norm(test_true_traffics - test_pred_traffics[sample_idx], np.inf, axis=1)
covered = np.sum(box_true_scores < conformal_quantile) / len(box_true_scores)

In [70]:
test_idx = 0
test_pred_traffics[sample_idx][test_idx].shape

(9867,)

In [72]:
from rsome import ro
from rsome import grb_solver as grb

model = ro.Model()

w = model.dvar(len(G.edges))
c = model.rvar(len(G.edges))

c_box_lb = test_pred_traffics[sample_idx][test_idx] - conformal_quantile
c_box_ub = test_pred_traffics[sample_idx][test_idx] + conformal_quantile
uset = (c_box_lb <= c, c <= c_box_ub)

model.minmax(c @ w, uset)
model.st(w <= 1)
model.st(w >= 0)
model.st(A @ w == b)

model.solve(grb)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-23
Being solved by Gurobi...
Solution status: 2
Running time: 0.0571s


In [73]:
model.get()

34473.57810163483

In [ ]:
def forecast_to_traffic(G, weathers):
    threshold_mmh = 0.1
    weathers[weathers < threshold_mmh] = 0

    x_coords = np.array(list(nx.get_node_attributes(G, "x").values()))
    y_coords = np.array(list(nx.get_node_attributes(G, "y").values()))

    x_coord_range = (np.min(x_coords), np.max(x_coords))
    y_coord_range = (np.min(y_coords), np.max(y_coords))

    # get rain at each node
    scaled_x = np.clip(((x_coords - x_coord_range[0]) / (x_coord_range[1] - x_coord_range[0])) * weathers.shape[1], 0, weathers.shape[1] - 1).astype(int)
    scaled_y = np.clip(((y_coords - y_coord_range[0]) / (y_coord_range[1] - y_coord_range[0])) * weathers.shape[2], 0, weathers.shape[2] - 1).astype(int)
    scaled_coords = np.vstack([scaled_x, scaled_y]).T

    rain_node_predictions = weathers[:,scaled_coords[:,0],scaled_coords[:,1]]

    # average between node endpoints to get rain along edges
    nodes_idx_to_idx = dict(zip(list(G.nodes), range(len(G.nodes))))
    edges = np.array([(nodes_idx_to_idx[e[0]], nodes_idx_to_idx[e[1]]) for e in list(G.edges)])
    rain_edge_predictions = np.concatenate([
        np.expand_dims(rain_node_predictions[:,edges[:,0]], axis=-1), 
        np.expand_dims(rain_node_predictions[:,edges[:,1]], axis=-1),
    ], axis=-1)

    travel_time = np.array(list(nx.get_edge_attributes(G, name="travel_time").values()))
    forecast_traffic = travel_time * np.exp(np.mean(rain_edge_predictions, axis=-1))

    return forecast_traffic